In [79]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
#read  data from 'Flights dataset.csv'
df = pd.read_csv('Flights dataset.csv')
#create new df , get 'ORIGIN_STATE_NM','ORIGIN_CITY_NAME','CARRIER','ARR_DELAY','DAY_OF_WEEK' from data and drop null
df = df[['ORIGIN_STATE_NM','ORIGIN_CITY_NAME','CARRIER','ARR_DELAY','DAY_OF_WEEK']].dropna()
# get delay flight number and create 'count' colummn as counter
df = df[df['ARR_DELAY'] > 0]
df['count']=1
#using groupby to filter data decrease the data number
df = df.groupby(['ORIGIN_STATE_NM','ORIGIN_CITY_NAME','CARRIER','DAY_OF_WEEK']).sum().reset_index()
# levels used for the hierarchical chart
levels = ['CARRIER', 'ORIGIN_CITY_NAME', 'ORIGIN_STATE_NM'] 
color_columns = ['count']
value_column = 'count'
#define the build_hierarchical_dataframe to create new data
def build_hierarchical_dataframe(df, levels, value_column, color_columns=None):
#create new dataframe 'df_all_trees' with 'id', 'parent', 'value', 'color'
    df_all_trees = pd.DataFrame(columns=['id', 'parent', 'value', 'color'])
#use for loop for the levels enum to plot the data in the df_all_trees
    for i, level in enumerate(levels):
        df_tree = pd.DataFrame(columns=['id', 'parent', 'value', 'color'])
        dfg = df.groupby(levels[i:]).sum()
        dfg = dfg.reset_index()
        df_tree['id'] = dfg[level].copy()
        if i < len(levels) - 1:
            df_tree['parent'] = dfg[levels[i+1]].copy()
        else:
            df_tree['parent'] = 'Total'
        df_tree['value'] = dfg[value_column]
        df_tree['color'] = dfg[color_columns]
        df_all_trees = df_all_trees.append(df_tree, ignore_index=True)
    total = pd.Series(dict(id='Total', parent='',
                              value=df[value_column].sum(),
                              color=df[color_columns].sum(), 
                          ))
    df_all_trees = df_all_trees.append(total, ignore_index=True)
    return df_all_trees
#use the previous function build_hierarchical_dataframe  implment attribute to create the Sunburst plot dataframe
df_all_trees = build_hierarchical_dataframe(df, levels, value_column, color_columns)

#Use graph_objects model to creat Sunburst graph with new dataframe df_all_trees
fig=go.Figure(go.Sunburst(
    labels=df_all_trees['id'],
    parents=df_all_trees['parent'],
    values=df_all_trees['value'],
    branchvalues='total',
    marker=dict(
        colors=df_all_trees['color'],
        colorscale='jet',
        showscale = True
        ),
    hovertemplate='<b>%{label} </b> <br> Count: %{value}</b> <br>Percent: %{percentRoot:.4%}</b>',
    maxdepth=2
    ))

#set up layout which set sunburst graph in center and creat title let the title in the top mid position
fig.update_layout(margin=dict(t=40, b=10, r=10, l=10),
       title ={
        'text': "Delay Flight Number And Percentage Under State, City and Carrier Condition(Sunburst.Ver)",
        'y':0.99,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'}
)
                 

fig.show()